In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from gensim.sklearn_api import LdaTransformer
import csv
import pandas as pd
from gensim.test.utils import datapath
from gensim.corpora import MmCorpus
from gensim.test.utils import get_tmpfile
import numpy as np

# Set up corpus and dictionary

In [0]:
path = '/content/drive/My Drive/sample_data/'
#out_path = '/content/drive/My Drive/LDA results/'
dictionary = Dictionary.load(path + 'GH_full_processed_Dictionary.dict')
corpus = MmCorpus(datapath(path + 'corpus_processed_GH_full.mm'))
texts = []
with open(path + 'GH_full_processed_corpus.csv', 'r') as f:
    reader = csv.reader(f)
    texts = list(reader)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
config_path = path + "GH_full_grid_search_log.csv"

def saveModelConfigs(model, coherence, u_mass, c_uci, c_npmi, path):
    with open(path, 'a') as f:
        writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([model.gensim_model.num_topics, "auto", coherence, u_mass, c_uci, c_npmi])


def fit_model(topicNum):
    model = LdaTransformer(id2word=dictionary, num_topics=topicNum, alpha='auto', eta="auto", iterations=400, random_state=2019)
    lda = model.fit(corpus)

    try:
      cm = CoherenceModel(model=lda.gensim_model, texts=texts, coherence='c_v')
      coherence = cm.get_coherence()
    except:
      coherence = -11111

    try:
      cm = CoherenceModel(model=lda.gensim_model, corpus=corpus, dictionary=dictionary, coherence='u_mass')
      u_mass = cm.get_coherence()
    except:
      u_mass = -11111

    try:
      cm = CoherenceModel(model=lda.gensim_model, texts=texts, coherence='c_uci')
      c_uci = cm.get_coherence()
    except:
      c_uci = -11111

    try:
      cm = CoherenceModel(model=lda.gensim_model, texts=texts, coherence='c_npmi')
      c_npmi = cm.get_coherence()
    except:
      c_npmi = -11111
    
    saveModelConfigs(lda, coherence, u_mass, c_uci, c_npmi, config_path)
    return lda.gensim_model

In [0]:
topic_num = []
for i in range(43, 50):
  topic_num.append(i)

for topic in topic_num:
  print("---------------------------topicNum = ", topic, "-----------------------------------")
  #for beta in betas:
  lda = fit_model(topic)
  temp_file = datapath("/content/drive/My Drive/sample_data/GH_full_saved_models/topic_" + str(topic) + ".lda")
  lda.save(temp_file)

---------------------------topicNum =  43 -----------------------------------


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


---------------------------topicNum =  44 -----------------------------------
---------------------------topicNum =  45 -----------------------------------
---------------------------topicNum =  46 -----------------------------------
---------------------------topicNum =  47 -----------------------------------
---------------------------topicNum =  48 -----------------------------------
